<a href="https://colab.research.google.com/github/Parth7/ML-for-finance/blob/main/NN_VAR_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib
import random
print(tf.__version__)

2.5.0


In [3]:
from google.colab import files
uploaded = files.upload()

Saving cummulative_data.xls to cummulative_data.xls


In [4]:
file = pd.read_excel('cummulative_data.xls') 
# Data for three-dimensional scattered points


#(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

# get per-feature statistics (mean, standard deviation) from the training set to normalize by
#train_mean = np.mean(train_features, axis=0)
#train_std = np.std(train_features, axis=0)
#train_features = (train_features - train_mean) / train_std

In [5]:
file = file.to_numpy()

In [6]:
file[:10]

array([[13.992114, 25.      , 10.      ],
       [13.930585, 25.      , 10.      ],
       [13.809978, 25.      , 10.      ],
       [13.911977, 25.      , 10.      ],
       [13.810281, 25.      , 10.      ],
       [13.961872, 25.      , 10.      ],
       [14.06189 , 25.      , 10.      ],
       [13.994377, 25.      , 10.      ],
       [13.922221, 25.      , 10.      ],
       [13.952919, 25.      , 10.      ]])

In [7]:
np.random.shuffle(file) 

In [8]:
file[:10]

array([[ 13.30079075,  75.        , -30.        ],
       [ 15.202598  , 125.        ,  30.        ],
       [ 13.44995436, 100.        , -10.        ],
       [ 17.98685907,  75.        ,  30.        ],
       [ 15.040808  ,  50.        ,  30.        ],
       [ 10.87649   ,  25.        ,  40.        ],
       [ 15.39652   , 125.        ,  40.        ],
       [ 14.78732236,  75.        , -10.        ],
       [ 15.361711  ,  50.        ,  50.        ],
       [ 15.67493   , 125.        ,  10.        ]])

In [9]:
file[:,0]

array([13.30079075, 15.202598  , 13.44995436, ..., 13.71242499,
       11.7928284 , 14.976642  ])

In [10]:
zdata = file[:,0]
xdata = file[:,1]
ydata = file[:,2]

data = np.array([xdata,ydata])
zd = np.array([zdata])

In [11]:
test_set_size = int(np.round(0.2*data.shape[1]));
train_set_size = data.shape[1] - (test_set_size);

In [12]:
x_train = data[:,:train_set_size]
y_train = zd[:,:train_set_size]

x_test = data[:,train_set_size:]
y_test = zd[:,train_set_size:]

In [13]:
x_train[:15]

array([[ 75., 125., 100., ..., 125., 125.,  50.],
       [-30.,  30., -10., ...,  30.,  10.,  40.]])

In [14]:
train_features = x_train

In [15]:
y_train.T.shape

(8000, 1)

In [16]:
x_train.T.shape

(8000, 2)

In [17]:
x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [31]:
#Define training hyperprameters.
batch_size = 50
num_epochs = 200
learning_rate = 0.01
size_hidden= 20

#Calculate some other hyperparameters based on data.  
batch_no = len(x_train) // batch_size  #batches
cols=x_train.shape[1] #Number of columns in input matrix
n_output=1

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :",device)
class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(cols, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
net = Net(cols, size_hidden, n_output)

Executing the model on : cuda:0


In [34]:
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
criterion = torch.nn.MSELoss(size_average=False)  # this is for regression mean squared loss

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [35]:
#Change to numpy arraay. 
X_train=x_train
y_train=y_train
X_test=x_test
y_test=y_test

In [36]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0
for epoch in range(num_epochs):
    #Shuffle just mixes up the dataset between epocs
    X_train, y_train = shuffle(X_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.FloatTensor(y_train[start:end]))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(outputs, torch.unsqueeze(labels,dim=1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([50, 1, 1])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 loss:  18917375.84375
Epoch 2 loss:  15262718.203125
Epoch 3 loss:  11936691.171875
Epoch 4 loss:  10954153.853515625
Epoch 5 loss:  10740353.29296875
Epoch 6 loss:  10930572.599609375
Epoch 7 loss:  10710319.775390625
Epoch 8 loss:  10959041.576171875
Epoch 9 loss:  10571915.67578125
Epoch 10 loss:  10701197.869140625
Epoch 11 loss:  10555605.8984375
Epoch 12 loss:  10615626.45703125
Epoch 13 loss:  10618047.30859375
Epoch 14 loss:  10657134.0703125
Epoch 15 loss:  10649601.623046875
Epoch 16 loss:  10576247.97265625
Epoch 17 loss:  10604817.84765625
Epoch 18 loss:  10526941.720703125
Epoch 19 loss:  10691716.099609375
Epoch 20 loss:  10584954.4453125
Epoch 21 loss:  10650212.8203125
Epoch 22 loss:  10495562.62890625
Epoch 23 loss:  10483724.650390625
Epoch 24 loss:  10493876.56640625
Epoch 25 loss:  10448219.423828125
Epoch 26 loss:  10613788.79296875
Epoch 27 loss:  10479097.68359375
Epoch 28 loss:  10558107.51953125
Epoch 29 loss:  10485847.451171875
Epoch 30 loss:  1048271

In [37]:
import pandas as pd
from sklearn.metrics import r2_score

X = Variable(torch.FloatTensor(X_train)) 
result = net(X)
pred=result.data[:,0].numpy()
print(len(pred),len(y_train))
r2_score(pred,y_train)

8000 8000


-938.1946533197736

In [28]:
import pandas as pd
from sklearn.metrics import r2_score
#This is a little bit tricky to get the resulting prediction.  
def calculate_r2(x,y=[]):
    """
    This function will return the r2 if passed x and y or return predictions if just passed x. 
    """
    # Evaluate the model with the test set. 
    X = Variable(torch.FloatTensor(x))  
    result = net(X) #This outputs the value for regression
    result=result.data[:,0].numpy()
  
    if len(y) != 0:
        r2=r2_score(result, y)
        print("R-Squared", r2)
        #print('Accuracy {:.2f}'.format(num_right / len(y)), "for a total of ", len(y), "records")
        return pd.DataFrame(data= {'actual': y, 'predicted': result})
    else:
        print("returning predictions")
        return result

In [29]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit( X_train, y_train )

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:

print('R2 for Train)', lm.score( X_train, y_train ))
print('R2 for Test (cross validation)', lm.score(X_test, y_test))

R2 for Train) 0.03218773200271918
R2 for Test (cross validation) 0.0283905623190871
